In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import numpy as np
import seaborn as sns
import datetime as dt
import cufflinks as cf
from IPython.display import display, HTML
from sodapy import Socrata
import streamlit as st
import mysql.connector
from sqlalchemy import create_engine

from pathlib import Path
import csv
import os

In [ ]:
def generarId (df, colmna):
    df = df.sort_values(by=colmna)
    n = 0
    nombre = "Id" + colmna
    df[nombre] = n
    for i in df[colmna].unique():
        n += 1
        df.loc[df[colmna] == i, nombre] = n
    return df

In [ ]:
def limpieza (df):
    translator = Translator()
    for i in df.columns:
        df.rename(columns={i:translator.translate(i, dest="es").text.title()}, inplace=True)
    df.drop(columns="Sin Nombre: 0", inplace=True)

    otro = pd.read_csv("./owid-co2-data.csv", delimiter=",", encoding="utf-8")
    otro = otro[["year","country","population","gdp"]]
    df = pd.merge(df, otro, left_on=["Año","País"], right_on=["year","country"], how="left")
    df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False), "Población"] = df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False)]["population"]
    df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False), "Pib"] = df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False)]["gdp"]
    df.drop(columns=["country", "year", "population","gdp"], inplace=True)

    df.dropna(subset=["Emisión De Co2"], inplace=True)

    for i in df.columns:
        if i == "Tipo_Energia":
            continue
        elif "Energ" in i:
            nom = i + " " + "TWh"
            df[nom] = df[i] / 3.412e+12

    return df

In [ ]:
def prueba(path):
    with os.scandir(path) as ficheros:
        for i in ficheros:
            directorio, extencion = os.path.splitext(i)
            if extencion == "":
                prueba(directorio)
                continue
            else:
                if extencion == ".txt" or extencion == ".csv":
                    ruta = directorio + extencion
                    arch = os.path.basename(ruta)
                    df = DataFrame2(ruta)
                    baseDeDatos(df,arch[:-4])